In [1]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score
import random
import operator
import copy


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df1= pd.read_csv('Phishing-Websites-Detection-master\Phishing.csv')
df1

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11050,1,-1,1,-1,1,1,1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11051,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11052,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1
11053,-1,-1,1,1,1,-1,-1,-1,1,-1,...,-1,1,1,1,1,-1,1,1,1,-1


In [3]:
# df1 = df.head(500)
# 

In [4]:
# df=pd.get_dummies(df1)
df1
# df1.rename(columns={'Result': 'Class'}, inplace=True)

df1['Result'] = df1['Result'].map({-1:0, 1:1})
df1['Result'].unique()

array([0, 1], dtype=int64)

In [5]:
class NeuralNetwork():
  def __init__(self, sizes, weights):
    self.sizes = sizes
    self.params = weights
  
  def sigmoid(self, x, derivative=False):
    if derivative:
      return (np.exp(-x))/((np.exp(-x)+1)**2)
    return 1/(1 + np.exp(-x))
    
  def softmax(self, x, derivative=False):
    # Numerically stable with large exponentials
    exps = np.exp(x - x.max())
    if derivative:
      return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
    return exps / np.sum(exps, axis=0)    
  
  def forward_pass(self, x_train):
    params = self.params
    
    # input layer activations becomes sample
    params['A0'] = x_train

    # input layer to hidden layer 1
    params['Z1'] = np.dot(params["W1"], params['A0'])
    params['A1'] = self.sigmoid(params['Z1'])

    # hidden layer 1 to hidden layer 2
    params['Z2'] = np.dot(params["W2"], params['A1'])
    params['A2'] = self.sigmoid(params['Z2'])

    # hidden layer 2 to output layer
    params['Z3'] = np.dot(params["W3"], params['A2'])
    params['A3'] = self.sigmoid(params['Z3'])

    return params['A3']



  def compute_accuracy(self, x_val, y_val):
    '''
        This function does a forward pass of x, then checks if the indices
        of the maximum value in the output equals the indices in the label
        y. Then it sums over each prediction and calculates the accuracy.
    '''
    predictions = []
    for x, y in zip(x_val, y_val):
      output = self.forward_pass(x)
      pred = np.argmax(output)
      predictions.append(pred == np.argmax(y))
        
    return np.mean(predictions)

  def prediction_value(self, x_val, y_val):
    '''
        This function does a forward pass of x, then checks if the indices
        of the maximum value in the output equals the indices in the label
        y. Then it sums over each prediction and calculates the accuracy.
    '''
    predictions = []
    for x, y in zip(x_val, y_val):
      output = self.forward_pass(x)
      pred = np.argmax(output)
      predictions.append(pred == np.argmax(y))
      
        
    return predictions


  def prediction_score(self, x_val, y_val):
    '''
        This function does a forward pass of x, then checks if the indices
        of the maximum value in the output equals the indices in the label
        y. Then it sums over each prediction and calculates the accuracy.
    '''
    outputs = []
    for x, y in zip(x_val, y_val):
      output = self.forward_pass(x)
      #pred = np.argmax(output)
      pred = np.where(output[0] > 0.5, 1, 0)


      outputs.append(pred)
      #predictions.append(pred == np.argmax(y))
   
    accuracy = accuracy_score(y_val, outputs)   
    return accuracy

In [6]:
class AG(object):
  def __init__(self, instance):
    self.instance = instance
    self.fitness_stat = []
    self.generation_stat = []
    self.time_stat = []
    self.mean_stat = []
    self.counter_time = 0
    self.counter = 0

  
  '''Executing the algorithm'''
  def run(self):
        
    ''' the initial population '''
    population = self.instance.initializePopulation()
    

    ''' General cycle of the genetic algorithm '''
    while 1:
      start = time.process_time()
    
      ''' Evaluation of the individuals of the population'''

      self.counter+=1  
      fitness_population = []
      mean_fitness= 0
      for individual in population:
        fitness = self.instance.getFitness(individual)
        val = {'fitness':fitness , 'individual' : individual}
        mean_fitness += fitness 
        fitness_population.append(val)
      
      self.mean_stat.append(mean_fitness/len(population))
      print("Generation", self.counter, "Accuracy",mean_fitness/len(population))

      '''record the statistics'''
      self.fitness_stat.append(self.instance.getFitnessStat())
      self.generation_stat.append(self.instance.getGenerationStat())
      self.time_stat.append(self.counter_time)


      ''' Best solution and check the exit criteria'''
      if self.instance.showAndCheck(fitness_population):
        break
      
      ''' Advance to the next town'''
      population = self.changePopulation(fitness_population);

      interval = time.process_time() - start
      self.counter_time  = self.counter_time + interval
      
      

  ''' Change from old population to the new one'''
  def changePopulation(self,fitness_population):
    ''' Selection of the best individuals (using the tournament method) '''
    parentsGenerator = self.instance.selectParents(fitness_population)
		
    
    allChildren = []
    ''' create new population '''
    while len(allChildren) < len(fitness_population):
      parents = next(parentsGenerator) # Next yielded pair of parents

      '''Probability for Crossing'''
      if random.random() > self.instance.getCrossThreshold():
        children = self.instance.crossover(parents)
      else:
        children = parents
      '''Probability for the mutation'''
      for child in children:
        if random.random() > self.instance.getMutationThreshold():
          allChildren.append(self.instance.mutate(child))
        else:
          allChildren.append(child)

    '''New population'''  
    return allChildren[:len(fitness_population)] # May exceed


  ''' print_result '''
  def get_best_individua(self):
    return self.instance.bestIndividual
  
  def print_best_result(self):
    print(f"Generation {self.instance.bestGeneration} Best Solution: {self.instance.bestIndividual} with fitness: {self.instance.bestFitness} ")

  ''' graban las estadisticas'''
  def result_statistics(self):
    plt.figure(figsize=(6,5))
    plt.plot(self.generation_stat, self.fitness_stat,'bo-')
    plt.xlabel('Generation ', fontsize=14)
    plt.ylabel('Best Solution', fontsize=14)
    plt.title( f"Mutation Rate.{self.instance.getMutationThreshold()} %,  Crossing Rate. {self.instance.getCrossThreshold()}%, Last generation {self.generation_stat[-1]}", fontsize=14)
    plt.show()
  
  def time_statistics(self):
    plt.figure(figsize=(6,5))
    plt.plot(self.generation_stat, self.time_stat,'bo-')
    plt.xlabel('Generation ', fontsize=14)
    plt.ylabel('Time ', fontsize=14)
    plt.title( " Time ", fontsize=14)
    plt.show()

  def mean_statistics(self):
    plt.figure(figsize=(6,5))
    plt.plot(self.generation_stat, self.mean_stat,'bo-')
    plt.xlabel('Generation ', fontsize=14)
    plt.ylabel('Accuracy ', fontsize=14)
    plt.title( " Accuracy  ", fontsize=14)
    plt.show()

In [7]:
class rules(object):

  ''' params '''

  def __init__(self, maxLoopsNum, error, populationSize, crossThreshold, mutationThreshold,X,y,structure):
    self.counter = 0
    self.maxLoopsNum = maxLoopsNum
    self.error = error
    self.populationSize = populationSize
    self.crossThreshold = crossThreshold
    self.mutationThreshold = mutationThreshold
    self.fitness_stat = 0
    self.generation_stat = 0 
    self.bestIndividual = list()
    self.bestFitness = 0
    self.bestGeneration = 0
    self.structure = structure
    self.X = X
    self.y = y

    

  def getMutationThreshold(self):
    return self.mutationThreshold

  def getCrossThreshold(self):
    return self.crossThreshold
    
  def getMaxLoopsNum(self):
    return self.maxLoopsNum

  def getFitnessStat(self):
    return self.fitness_stat

  def getGenerationStat(self):
    return self.generation_stat
    
  def getLastIndividual(self):
    return self.bestIndividual

  ''' Initial population '''
  def initializePopulation(self):
    population = [];
    for i in range(self.populationSize):
      population.append(self.generate_weights_individu(self.structure))
    return population
  
  def getFitness(self, individual):
    nn =   NeuralNetwork(sizes=self.structure, weights = individual)
    score = nn.prediction_score(self.X, self.y)
    return  score

  def showAndCheck(self, fitness_population):
    self.counter += 1
    fitness_population.sort(key=operator.itemgetter('fitness'))
    # Printing Generation Fitness
    #print("Generation", self.counter, "Best solution:", fitness_population[-1]['fitness'])
      
    # Saving The Stats
    self.fitness_stat = fitness_population[-1]['fitness']
    self.generation_stat = self.counter
    best = fitness_population[-1]  # best Individual


    # best solution
    if (fitness_population[-1]['fitness'] >= self.bestFitness): 
      self.bestIndividual = fitness_population[-1]['individual']
      self.bestFitness = fitness_population[-1]['fitness']
      self.bestGeneration = self.counter

    check = (self.counter > self.maxLoopsNum) or (fitness_population[-1]['fitness']>= self.error)
    return check

  def selectParents(self, fitness_population):
    # Construct a iterator here
    # Use Tournament Selection
    while 1:
      parent1 = self.tournament(fitness_population)
      parent2 = self.tournament(fitness_population)
      yield (parent1, parent2)
      
  def crossover(self, parents):
    parent1, parent2 = parents
    father_1, father_2 = parents
    chlid_1 = {
      'W1': father_1['W1'],
      'W2': father_2['W2'] ,
      'W3': father_2['W3']
    }

    chlid_2 = {
      'W1': father_2['W1'],
      'W2': father_2['W2'] ,
      'W3': father_1['W3']
    }
    return (chlid_1,  chlid_2)
    
  def mutate(self, child):
    child_m = copy.deepcopy(child)
    leyers =  ['W1','W2', 'W3']  
    chosen_layer_index = random.randrange(0,2)
    chosen_layer = leyers[chosen_layer_index]

    random_weights = np.random.uniform(-1,1)

    #get x an y from shape 
    x_max, y_max = child[chosen_layer].shape

    # index to mutate
    x = random.randrange(0,x_max-1)
    y = random.randrange(0,y_max-1)

    child_m[chosen_layer][x][y] = random_weights
  
    return child_m
  
  def tournament(self, fitness_population):

    f1 = random.randint(0, len(fitness_population) - 1)
    f2 = random.randint(0, len(fitness_population) - 1)
    fit1 =fitness_population[f1]['fitness'] , 
    ch1 = fitness_population[f1]['individual']
    fit2 =fitness_population[f2]['fitness'] , 
    ch2 = fitness_population[f2]['individual']

    return ch1 if fit1 > fit2 else ch2

  def generate_weights_individu(self, sizes):
    
    # number of nodes in each layer
    input_layer=sizes[0]
    hidden_1=sizes[1]
    hidden_2=sizes[2]
    output_layer=sizes[3]
    
    params = {
      'W1':np.random.uniform(-10,10, size=(hidden_1,input_layer)),
      'W2':np.random.uniform(-10,10, size=(hidden_2, hidden_1)) ,
      'W3':np.random.uniform(-10,10, size=(output_layer, hidden_2))
    }
    
    return params 


In [8]:

x = df1.drop('Result', inplace=False, axis=1) #remove 'target' column from input features
y = df1['Result'] #stores target (1 or 0) in a separate array

In [9]:
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.1, random_state = 0)

X_train = X_train.reset_index(drop=True) 
y_train = y_train.reset_index(drop=True)

X_train = np.array(X_train, dtype=float)
y_train = np.array(y_train, dtype=float)

y_train = np.array(y_train)


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

# Define the grid of values
penalty = ['l1', 'l2']
C = [0.8, 0.9, 1.0]
tol = [0.01, 0.001 ,0.0001]
max_iter = [100, 150, 200, 250]

# Create a dictionary where tol and max_iter are keys and the lists of their values are the corresponding values
param_grid = dict(penalty=penalty, C=C, tol=tol, max_iter=max_iter)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

random_model = RandomizedSearchCV(estimator=logreg, param_distributions=param_grid, cv=5)

# Fit random_model to the data
random_model_result = random_model.fit(X_train, y_train)

# Summarize results
best_score, best_params = random_model_result.best_score_, random_model_result.best_params_
print("Best score: %.2f using %s" % (best_score*100., best_params))

Best score: 92.92 using {'tol': 0.01, 'penalty': 'l2', 'max_iter': 100, 'C': 0.9}


In [15]:
# gradeFunction, maxLoopsNum, error, populationSize, crossThreshold, mutationThreshold
maxLoopsNum = 200
error = 0.95
populationSize = 100
crossThreshold = 0.8
mutationThreshold =  0.01

ag =  AG(rules(maxLoopsNum, error,  populationSize, crossThreshold, mutationThreshold, X_train, y_train,[30, 50, 2, 1]))
ag.run() 

Generation 1 Accuracy 0.5045321137802792
Generation 2 Accuracy 0.5341461453412406
Generation 3 Accuracy 0.5606181525781486
Generation 4 Accuracy 0.5729470298522469
Generation 5 Accuracy 0.5846567494220526
Generation 6 Accuracy 0.5866760478440046
Generation 7 Accuracy 0.5963986330284449
Generation 8 Accuracy 0.6083194290883507
Generation 9 Accuracy 0.6115428686300132
Generation 10 Accuracy 0.6433239521559954
Generation 11 Accuracy 0.6631601165946323
Generation 12 Accuracy 0.6840456327269071
Generation 13 Accuracy 0.7077364559252188
Generation 14 Accuracy 0.7235239722585183
Generation 15 Accuracy 0.7294954266760478
Generation 16 Accuracy 0.7367514323047542
Generation 17 Accuracy 0.7382189164740174
Generation 18 Accuracy 0.743722987234898
Generation 19 Accuracy 0.7480892552015278
Generation 20 Accuracy 0.7526324253693837
Generation 21 Accuracy 0.7582128857171577
Generation 22 Accuracy 0.7605799577847017
Generation 23 Accuracy 0.7638044024525078
Generation 24 Accuracy 0.764857774650719
Gen

In [17]:

x_test = np.array(x_test, dtype=float)
y_test = np.array(y_test, dtype=float)



In [18]:
y_test = np.array(y_test)


In [19]:
weights =  ag.get_best_individua()
nn = NeuralNetwork(sizes=[30, 10, 5, 1], weights = weights)
print(nn.prediction_score(x_test,y_test))

0.8553345388788427


NameError: name 'nn' is not defined